In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-toy-dataset/covid_toy.csv


In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('/kaggle/input/covid-toy-dataset/covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train,X_test,Y_train,Y_test=train_test_split(df.drop(columns=['has_covid']), df['has_covid'],
                                                test_size=0.2)

In [8]:
X_train.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,age,gender,fever,cough,city
94,79,Male,NaN,Strong,Kolkata
34,74,Male,102.0,Mild,Mumbai
4,65,Female,101.0,Mild,Mumbai
6,14,Male,101.0,Strong,Bangalore
19,42,Female,NaN,Strong,Bangalore


In [9]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [10]:
from sklearn.compose import ColumnTransformer

In [11]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),  # Impute missing values in 'fever'
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),  # Ordinal encoding for 'cough'
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])  # Use 'sparse_output=False' instead of 'sparse'
], remainder='passthrough')  # Keep other columns unchanged

In [12]:
transformer.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tnf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [13]:
transformer.transform(X_train).shape

(80, 7)

In [14]:
transformer.transform(X_test).shape

(20, 7)